In [ ]:
import polars as pl
import pandas as pd
import networkx as nx
import os
import rtsvg
rt = rtsvg.RACETrack()

#
# Select By Color doesn't work if there's a cloud with multiple
# colors within it...  if the cloud is only single color, it works...
#

edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df            = pl.DataFrame(_lu_)
df            = df.with_columns(pl.col('fm').cast(pl.String), pl.col('to').cast(pl.String))
g             = rt.createNetworkXGraph(df, [('fm','to')])
_pos_         = nx.spring_layout(g)
_relates_     = [('fm','to')]
_node_colors_ = {}
community_i   = 0
for _community_ in nx.community.louvain_communities(g):
    community_i += 1
    for _node_ in _community_: _node_colors_[_node_] = rt.co_mgr.getColor(community_i)
_ln_params_ = {'relationships':_relates_, 'pos':_pos_, 'color_by':'gp', 'node_color':_node_colors_, 'draw_labels':False}
_l_  = rt.link    (df, track_state=True, **_ln_params_)
_ln_ = rt.linkNode(df, track_state=True, **_ln_params_)
_l_._repr_svg_(), _ln_._repr_svg_()
#rt.tile([_l_, _ln_], spacer=10)
_igl_       = rt.interactiveGraphLayout(df, _ln_params_, w=600, h=600)
_igl_